In [ ]:
#pip install requests

In [2]:
import requests
import json
import time
from datetime import datetime, timedelta

In [3]:
def to_seconds(days=0, hours=0, minutes=0, seconds=0):
    hours+=days*24
    minutes+=hours*60
    seconds+=minutes*60
    return seconds

In [4]:
now = datetime.now().timestamp()
print(now)

1670602266.776025


In [ ]:
###################### FTX #################################

In [ ]:
def get_apr(start_time, end_time, future):
    
    #Queries funding rate data in the given period
    params = {'start_time':start_time, 'end_time':end_time,'future':future}
    response = requests.get("https://ftx.com/api/funding_rates", params=params)
    data=response.json()
    
    #Calculates the average hourly rate
    total=0
    for result in data['result']:
        total+=result['rate']
    data_length=len(data['result'])
    average=total/data_length
    
    #Calculates the yearly rate (APR)
    APR=average*24*365
    return str(data['result'][0]['future']+': {:.2%}'.format(APR))

In [ ]:
now=time.time()
params = {'start_time':str(now-to_seconds(days=3)), 'end_time':str(now),'future':'GST-PERP'}
response = requests.get("https://ftx.com/api/funding_rates", params=params)
data=response.json()
print(json.dumps(data, indent=2))

In [ ]:
total=0
for result in data['result']:
    total+=result['rate']

data_length=len(data['result'])
average=total/data_length
APR=average*24*365
print(data_length)
print('{:.2%}'.format(APR))

In [ ]:
#Get the average APR from all FTX perpetual futures on the last 3 days

#Part(1/2) 

#Gets funding rates from all perpetual futures in the last hour
#It's a way to get only the perpetual futures, as delivery futures don't have funding rates
params = {'start_time':str(now-to_seconds(hours=1)), 'end_time':str(now)}
response = requests.get("https://ftx.com/api/funding_rates", params=params)
data=response.json()
print(json.dumps(data, indent=2))

In [ ]:
#Part(2/2)

#Gets the name from all futures
futures=[]
for result in data['result']:
    futures.append((result['future']))
futures.sort()

#Prints the implied APR from each future
for future in futures:
    print(get_apr(str(now-to_seconds(days=3)), str(now), future))

In [ ]:
#Gets the APR from last 7 days
for future in futures:
    print(get_apr(str(now-to_seconds(days=7)), str(now), future))

In [ ]:
#Gets the APR from last 30 days

for future in futures:
    print(get_apr(str(now-to_seconds(days=30)), str(now), future))

Next Steps:

* Create a more user friendly way to define the timeframe
* Use FTX pagination and get data from timeframes greater than supported by default
* Order by APR
* Study the use of asynchronous calls to improve the speed of the algorithm
* Store the data so that when doing different timeframes, the overlapping time doesn't need to be queried repeatedly

In [ ]:
######################## OKX ###############################

In [ ]:
def get_okx_apr(before, after, coin):
    
    params = {'ccy': coin, 'before': before, 'after': after}
    response = requests.get("https://okx.com/api/v5/asset/lending-rate-history", params=params)
    data=response.json()
    
    #Calculates the average yearly rate
    total=0
    for datapoint in data['data']:
        total+=float(datapoint['rate'])
    data_length=len(data['data'])
    
    APR=total/data_length
    return str(coin+': {:.2%}'.format(APR))

In [ ]:
#Test for BTC

before=str(int(now)-to_seconds(days=4))+'000'
after=str(int(now)-to_seconds(days=0))+'000'
apr = get_okx_apr(before, after, 'BTC')
print(apr)


In [ ]:
#Gets all the currencies available in OKX for stop trading

params = {'instType': 'SPOT'}
response = requests.get("https://okx.com/api/v5/market/tickers", params=params)
print(response)
data=response.json()
print(len(data['data']))
currencies=[]
for datapoint in data['data']:
    base=datapoint['instId'].split('-')[0]
    if base not in currencies:
        currencies.append(base)
currencies.sort()
print(currencies)

In [ ]:
#Gets the interest rates on all coins available to borrow, based on an average of the last 4 days

before=str(int(now)-to_seconds(days=4))+'000'
after=str(int(now)-to_seconds(days=0))+'000'
for currency in currencies:
    try:
        apr = get_okx_apr(before, after, currency)
        print(apr)
    except:
        pass

In [ ]:
#response = requests.get("https://okx.com/api/v5/asset/lending-rate-history")
#data=response.json()
#print(json.dumps(data, indent=2))
#print(len(data['data']))

In [ ]:
####################### KUCOIN MARGIN ################################

In [5]:
BASE_KUCOIN='https://api.kucoin.com'

#response = requests.get(BASE_KUCOIN+'/api/v1/margin/market?currency=BTC')
#data=response.json()
#print(json.dumps(data, indent=2))

In [8]:
#Gets all the currencies on Kucoin that have margin trading availabe
#response = requests.get(BASE_KUCOIN+'/api/v1/currencies')
response = requests.get('https://api.kucoin.com/api/v1/currencies')
data=response.json()
currencies = []
for datapoint in data['data']:
    if datapoint['isMarginEnabled']:
        currencies.append(datapoint['currency'])

currencies.sort()
print(currencies)        

['1INCH', 'AAVE', 'ADA', 'AIOZ', 'ALGO', 'ALICE', 'ANC', 'ANKR', 'ANT', 'APE', 'API3', 'APT', 'AR', 'ARPA', 'ATOM', 'AUDIO', 'AVAX', 'AXS', 'BAT', 'BCH', 'BCHSV', 'BNB', 'BTC', 'BTT', 'C98', 'CAKE', 'CELO', 'CHR', 'CHZ', 'CKB', 'CLV', 'COMP', 'CRO', 'CRV', 'CTSI', 'DAO', 'DAR', 'DASH', 'DGB', 'DODO', 'DOGE', 'DOT', 'DYDX', 'EGLD', 'ELON', 'ENJ', 'ENS', 'EOS', 'ERN', 'ETC', 'ETH', 'ETHW', 'FIL', 'FITFI', 'FLOW', 'FLUX', 'FRONT', 'FTM', 'FXS', 'GAL', 'GALAX', 'GLMR', 'GMT', 'GRT', 'HBAR', 'ICP', 'ILV', 'IMX', 'INJ', 'IOST', 'IOTA', 'IOTX', 'JASMY', 'JST', 'KAVA', 'KCS', 'KDA', 'KSM', 'LINA', 'LINK', 'LPT', 'LRC', 'LTC', 'LTO', 'MANA', 'MASK', 'MATIC', 'MKR', 'MOVR', 'MXC', 'NEAR', 'NEO', 'NFT', 'NKN', 'OCEAN', 'OGN', 'OMG', 'ONE', 'OP', 'ORN', 'PEOPLE', 'POLS', 'POND', 'PYR', 'QI', 'QNT', 'REN', 'REQ', 'RLC', 'RNDR', 'ROSE', 'RSR', 'RUNE', 'SAND', 'SHIB', 'SKL', 'SLP', 'SNX', 'SOL', 'SOS', 'STORJ', 'STX', 'SUN', 'SUPER', 'SUSHI', 'SXP', 'THETA', 'TLM', 'TRX', 'UMA', 'UNI', 'USDC', 'USDD'

In [7]:
#For each currency
    #Get the price of the currency
    #Get the lending offers
    #See at which interest rate it has at least X dollars available for trade - print that rate - 
        #otherwise print either nothing or unavailable

USD_threshold=2000

for currency in currencies:
    response = requests.get(BASE_KUCOIN+'/api/v1/prices?currencies='+currency)
    data=response.json()
    price=float(data['data'][currency])
    response = requests.get(BASE_KUCOIN+'/api/v1/margin/market?currency='+currency)
    data=response.json()
    
    borrowable=False
    total=0
    for datapoint in data['data']:
        total+=float(datapoint['size'])
        if total*price > USD_threshold:
            APR = float(datapoint['dailyIntRate'])*365
            borrowable=True
            break
    if borrowable:
        print(currency+': '+': {:.2%}'.format(APR))
    else:
        print(currency+': '+'Not available')

1INCH: : 6.21%
AAVE: : 1.46%
ADA: : 5.11%
AIOZ: : 12.41%
ALGO: : 15.33%
ALICE: : 2.19%
ANC: : 32.85%
ANKR: : 3.65%
ANT: : 4.01%
APE: : 7.30%
API3: : 6.94%
APT: : 1.83%
AR: : 0.73%
ARPA: : 8.03%
ATOM: : 7.30%
AUDIO: : 14.60%
AVAX: : 2.19%
AXS: : 18.98%
BAT: : 0.37%
BCH: : 2.55%
BCHSV: : 2.19%
BNB: : 1.09%
BTC: : 1.09%
BTT: : 8.03%
C98: : 16.06%
CAKE: : 9.86%
CELO: : 7.67%
CHR: : 5.11%
CHZ: : 0.73%
CKB: : 2.55%
CLV: : 0.73%
COMP: : 3.65%
CRO: : 1.09%
CRV: : 17.52%
CTSI: Not available
DAO: : 11.68%
DAR: : 27.38%
DASH: : 3.65%
DGB: : 0.73%
DODO: : 1.09%
DOGE: : 5.47%
DOT: : 5.47%
DYDX: : 2.19%
EGLD: : 12.04%
ELON: : 0.73%
ENJ: : 0.37%
ENS: : 5.47%
EOS: : 6.21%
ERN: : 8.03%
ETC: : 0.73%
ETH: : 3.65%
ETHW: Not available
FIL: : 5.11%
FITFI: Not available
FLOW: : 10.95%
FLUX: : 6.21%
FRONT: : 2.92%
FTM: : 8.03%
FXS: : 11.68%
GAL: : 9.49%
GALAX: : 4.38%
GLMR: : 1.46%
GMT: : 0.73%
GRT: : 13.14%
HBAR: : 4.75%
ICP: : 7.30%
ILV: : 10.95%
IMX: : 0.73%
INJ: : 17.88%
IOST: : 8.03%
IOTA: : 0.73%
IOTX: 

In [ ]:
############### KUCOIN FUTURES ####################

#Still on exploratory phase

BASE_KUCOIN_FUTURES = 'http://api-futures.kucoin.com'
response = requests.get(BASE_KUCOIN_FUTURES+'/api/v1/contracts/active')
data=response.json()
#print(json.dumps(data, indent=2))


In [ ]:
response = requests.get(BASE_KUCOIN_FUTURES+'/api/v1/interest/query?symbol=.LINKINT8H')

data=response.json()
#print(json.dumps(data, indent=2))

In [ ]:
############### COIN SETS ###############

In [ ]:
SET1_FTX=['ALGO',
           'ALICE',
           'ATLAS',
           'ATOM',
           'BAL',
           'BTC',
           'CAKE',
           'CLV',
           'CRO',
           'CRV',
           'CVX',
           'FLM',
           'FLOW',
           'FXS',
           'KAVA',
           'KSM',
           'PERP',
           'PUNDIX',
           'ROSE',
           'STX',
            'VET'
           ]

SET1_OKX=['ATOM',
            'CLV',
            'CRO',
            'FLM',
           'KSM',
           'PERP']

SET1_KCS=['ALICE',
          'AXS',
          'CAKE',
         'CLV',
         'CRO',
         'CRV',
         'FLOW',
          'GLMR',
          'KAVA',
          'MOVR',
         'ROSE',
          'SLP',
          'STX',
         'VET',]

In [ ]:
apr=get_apr(str(now-to_seconds(days=60)), str(now-to_seconds(days=30)), "OKB-PERP")
print(apr)

In [ ]:
def get_ftx_futures(coin_set, days):

    futures=[]
    for coin in coin_set:
        futures.append(coin+'-PERP')
    futures.sort()

    #Prints the implied APR from each future
    for future in futures:
        print(get_apr(str(now-to_seconds(days=days)), str(now), future))

In [ ]:
get_ftx_futures(SET1_FTX, 3)

In [ ]:
def get_okx_rates(coin_set, days):
    
    before=str(int(now)-to_seconds(days=days))+'000'
    after=str(int(now)-to_seconds(days=0))+'000'
    for coin in coin_set:
        apr = get_okx_apr(before, after, coin)
        print(apr)

In [ ]:
get_okx_rates(SET1_OKX, 4)

In [ ]:
def get_kcs_margin(coin_set):
    

In [ ]:
def query_all(coin, days):
    print("OKX")
    before=str(int(now)-to_seconds(days=days))+'000'
    after=str(int(now)-to_seconds(days=0))+'000'   
    okx_apr = get_okx_apr(before, after, coin)
    print(okx_apr)
    print("FTX")
    future=coin+"-PERP"
    ftx_apr=get_apr(str(now-to_seconds(days=days)), str(now), future)
    print(ftx_apr)

In [ ]:
query_all("ETHW", 3)

In [ ]:
def get_kcs_price(currency):
    response = requests.get(BASE_KUCOIN+'/api/v1/prices?currencies='+currency)
    data=response.json()
    price=float(data['data'][currency])
    return price

In [ ]:
USD_threshold=2000

for currency in currencies:
    price
    response = requests.get(BASE_KUCOIN+'/api/v1/margin/market?currency='+currency)
    data=response.json()
    
    borrowable=False
    total=0
    for datapoint in data['data']:
        total+=float(datapoint['size'])
        if total*price > USD_threshold:
            APR = float(datapoint['dailyIntRate'])*365
            borrowable=True
            break
    if borrowable:
        print(currency+': '+': {:.2%}'.format(APR))
    else:
        print(currency+': '+'Not available')

In [ ]:
import urllib.parse as urlparse
from urllib.parse import urlencode

def add_url_params(url, params):
    #https://stackoverflow.com/questions/2506379/add-params-to-given-url-in-python
    url_parts = list(urlparse.urlparse(url))
    query = dict(urlparse.parse_qsl(url_parts[4]))
    query.update(params)

    url_parts[4] = urlencode(query)

    final_url=urlparse.urlunparse(url_parts)
    return final_url

url = '/api/v1/position'
params = {'symbol':'XBTUSDM'}
add_url_params(url, params)

In [ ]:
import time
import base64
import hmac
import hashlib
import requests

def kucoin_requests_get(endpoint):
    #https://docs.kucoin.com/futures/#authentication

    api_key = "633401973bcee200014177df"
    api_secret = "0d3c79c6-bbe1-49c1-9614-f8dc08b22736"
    api_passphrase = "unforgiven"
    url = 'https://api-futures.kucoin.com'+endpoint
    now = int(time.time() * 1000)
    str_to_sign = str(now) + 'GET' + endpoint
    signature = base64.b64encode(
        hmac.new(api_secret.encode('utf-8'), str_to_sign.encode('utf-8'), hashlib.sha256).digest())
    passphrase = base64.b64encode(hmac.new(api_secret.encode('utf-8'), api_passphrase.encode('utf-8'), hashlib.sha256).digest())
    headers = {
        "KC-API-SIGN": signature,
        "KC-API-TIMESTAMP": str(now),
        "KC-API-KEY": api_key,
        "KC-API-PASSPHRASE": passphrase,
        "KC-API-KEY-VERSION": "2"
    }
    response = requests.request('get', url, headers=headers)
    return response

url='/api/v1/position'
params={'symbol':'XBTUSDM'}
response=kucoin_requests_get(add_url_params(url,params))
print(response.status_code)
print(response.json())

In [ ]:
#This queries the history of the account rather than historical data
#before=str(int(now)-to_seconds(days=4))+'000'
#after=str(int(now)-to_seconds(days=0))+'000'
#params={'symbol':'TT', 'startAt':before, 'endAt':after}
url='/api/v1/funding-history'
params={'symbol':'AXSUSDTM'}
response=kucoin_requests_get(add_url_params(url,params))
print(response.status_code)
print(response.json())

In [ ]:
response=kucoin_requests_get('/api/v1/contracts/active')
print(response.status_code)
data=response.json()
print(json.dumps(data, indent=2))

In [ ]:
import urllib.parse as urlparse
from urllib.parse import urlencode

parsed_url

url = 'https://api-futures.kucoin.com/api/v1/position?symbol=XBTUSDM'
params = {'symbol':'XBTUSDM'}
url_parts = list(urlparse.urlparse(url))
query = dict(urlparse.parse_qsl(url_parts[4]))
query.update(params)

url_parts[4] = urlencode(query)

print(urlparse.urlunparse(url_parts))